In [1]:
# third-party
import pandas as pd
import scipy
import pickle
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

# local
from get_data import transform_overview_on_target, transform_overview_on_overall
from performance import get_breath_parameters_performance
from delay_performance import normality_test
from performance import bland_altman_plot, get_relative_errors
from get_data import get_data_by_id_activity
from constants import CATEGORICAL_PALETTE


In [2]:
with open('Results/results.pickle', 'rb') as file:
    overview = pickle.load(file)

overview_middle = {}
for id in overview.keys():
    overview_middle[id] = {key: value for key, value in overview[id].items() if key in ['SNBm', 'UALm', 'UARm']}

# remove middle activities
for id in overview.keys():
    del overview[id]["SNBm"]
    del overview[id]["UALm"]
    del overview[id]["UARm"]

In [3]:
acquisition_folderpath = 'Aquisicao'
data, _ = get_data_by_id_activity(acquisition_folderpath)


Getting data for participants...
 --------- 7OYX ---------------
 --------- NO15 ---------------
 --------- G8B7 ---------------
 --------- EPE2 ---------------
 --------- HAK8 ---------------
 --------- 1BST ---------------
 --------- 83J1 ---------------
 --------- QMQ7 ---------------
 --------- 9TUL ---------------
 --------- FTD7 ---------------
 --------- Y6O3 ---------------
 --------- 2QWT ---------------
 --------- F9AF ---------------
 --------- P4W9 ---------------
 --------- W8Z9 ---------------
 --------- D4GQ ---------------


In [4]:
def plot_resp_param(data, overview, id, activity, sensor, sampling_freq=100):

    sensor_dict = {"MAG": "mag", "PZT": "pzt"}

    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # plot sensor and airflow signals
    fig.add_trace(go.Scatter(x=np.linspace(1, len(data[id][activity][sensor_dict[sensor]]),len(data[id][activity][sensor_dict[sensor]]))/sampling_freq, 
                             y=data[id][activity][sensor_dict[sensor]], 
                             name=sensor))
    fig.add_trace(go.Scatter(x=np.linspace(1, len(data[id][activity]['airflow']),len(data[id][activity]['airflow']))/sampling_freq,
                             y=data[id][activity]['airflow'], 
                            name="Airflow", line={"color":"white"}), secondary_y=True)

    # add FR events
    peak_valley = {"TP_i": "valleys", "TP_e": "peaks", "FN_i": "valleys", "FN_e": "peaks", "FP_i": "valleys", "FP_e": "peaks"}

    for event in ["TP_i", "TP_e"]:
        if event == "TP_i":
            symbol = "triangle-up"
        else:
            symbol = "triangle-down"
        fig.add_trace(go.Scatter(x=np.array(overview[id][activity][sensor][event])/sampling_freq, y=data[id][activity][sensor_dict[sensor]][overview[id][activity][sensor][event]],
                            mode="markers",
                            marker = {
                                'symbol': symbol,
                                'size': 10
                            },
                            name=f"TP {peak_valley[event]}"))
        
        fig.add_trace(go.Scatter(x=np.array(overview[id][activity]["Airflow"][peak_valley[event]])/sampling_freq, y=data[id][activity]['airflow'][overview[id][activity]["Airflow"][peak_valley[event]]],
                        mode="markers",
                        marker = {
                                'color': 'white',
                                'symbol': symbol,
                                'size': 10
                            },
                        name=f"ref {peak_valley[event]}",), secondary_y=True)
    
    for event in ["FN_i", "FN_e"]:
        fig.add_trace(go.Scatter(x=np.array(overview[id][activity][sensor][event])/sampling_freq, y=data[id][activity][sensor_dict[sensor]][overview[id][activity][sensor][event]],
                            mode="markers",
                            name=f"FN {peak_valley[event]}"))
        
    for event in ["FP_i", "FP_e"]:
        fig.add_trace(go.Scatter(x=np.array(overview[id][activity][sensor][event])/sampling_freq, y=data[id][activity][sensor_dict[sensor]][overview[id][activity][sensor][event]],
                            mode="markers",
                            name=f"FP {peak_valley[event]}"))


    TP_peaks_tuple = [(event, True, "peak") for event in overview[id][activity][sensor]["TP_e"]]
    TP_valleys_tuple = [(event, True, "valley") for event in overview[id][activity][sensor]["TP_i"]]
    FN_peaks_tuple = [(event, False, "peak") for event in overview[id][activity][sensor]["FN_e"]]
    FN_valleys_tuple = [(event, False, "valley") for event in overview[id][activity][sensor]["FN_i"]]
    extrema = sorted(TP_peaks_tuple + TP_valleys_tuple + FN_peaks_tuple + FN_valleys_tuple)

    
    extrema_pairs_peaks = np.asarray([
        (extrema[i][0], extrema[i+1][0],
            all((extrema[i][1], extrema[i+1][1])))
        for i in range(0, len(extrema)-1)
        if extrema[i][2] == "peak" and extrema[i+1][2] == "valley"
    ])

    extrema_pairs_valleys = np.asarray([
        (extrema[i][0], extrema[i+1][0],
            all((extrema[i][1], extrema[i+1][1])))
        for i in range(0, len(extrema)-1)
        if extrema[i][2] == "valley" and extrema[i+1][2] == "peak"
    ])

    extrema_pairs_peaks = extrema_pairs_peaks[extrema_pairs_peaks[:,-1] != 0]
    extrema_pairs_valleys = extrema_pairs_valleys[extrema_pairs_valleys[:,-1] != 0]

    for i,ind in enumerate(extrema_pairs_peaks):
        try:
            exp_duration = overview[id][activity][sensor]["tE (s)"][i]
            fig.add_vrect(x0=ind[0]/sampling_freq, x1=ind[0]/sampling_freq+exp_duration, fillcolor=CATEGORICAL_PALETTE[0], 
                            opacity=0.2, line_width=0.2, line_color='white', annotation_text="exp", annotation_position="top left",)
        except Exception as e:
            print(e)

    for i, ind in enumerate(extrema_pairs_valleys):  
        try:
            insp_duration = overview[id][activity][sensor]["tI (s)"][i]
            fig.add_vrect(x0=ind[0]/sampling_freq, x1=ind[0]/sampling_freq+insp_duration, fillcolor=CATEGORICAL_PALETTE[1], 
                            opacity=0.2, line_width=0.2, line_color='white', annotation_text="insp", annotation_position="bottom left",)
        except Exception as e:
            print(e)
                
    fig.update_xaxes(showgrid=False)
    fig.update_yaxes(showgrid=False)
    fig.show()

    print(f'sensor: {overview[id][activity][sensor]["tI (s)"]} | airflow: {overview[id][activity][sensor]["tI airflow (s)"]}')
    

In [5]:

plot_resp_param(data, overview, id="EPE2", activity="SGB", sensor="MAG")


sensor: [5.49 4.48 3.76 4.61 4.02 3.96] | airflow: [4.65 4.44 4.52 5.04 4.69 4.56]


In [6]:
plot_resp_param(data, overview, id="2QWT", activity="TR", sensor="MAG")

sensor: [2.81 3.88 3.21 3.51 3.28 3.21 2.77 3.36 3.04] | airflow: [2.79 2.89 2.97 3.   2.82 2.94 2.57 3.09 2.71]


### Activity-specific only RP analysis

In [7]:
# Transform overview from participant specific into all participants

overview_all_participants = transform_overview_on_target(overview, target="Activity")
overview_all_participants.keys()

dict_keys(['SNB', 'SGB', 'MIXB', 'STNB', 'MCH', 'SQT', 'AAL', 'AAR', 'ALL', 'ALR', 'UAL', 'UAR', 'SE', 'SS', 'TR'])

In [8]:
performance_all_participants = get_breath_parameters_performance(overview_all_participants, target="Activity")
performance_all_participants[performance_all_participants["Sensor"]=="MAG"]

,Activity,Sensor,MAE Ti (s),MRE Ti (%),MAE Te (s),MRE Te (%),MAE Tb (s),MRE Tb (%)
0,SNB,MAG,0.07 $\pm$ 0.11,3.18 $\pm$ 4.80,0.07 $\pm$ 0.11,2.75 $\pm$ 4.22,0.06 $\pm$ 0.10,1.34 $\pm$ 2.18
2,SGB,MAG,0.49 $\pm$ 0.54,10.89 $\pm$ 12.36,0.49 $\pm$ 0.56,8.73 $\pm$ 10.03,0.34 $\pm$ 0.55,3.39 $\pm$ 5.54
4,MIXB,MAG,0.11 $\pm$ 0.14,3.93 $\pm$ 4.98,0.11 $\pm$ 0.15,4.09 $\pm$ 5.24,0.09 $\pm$ 0.12,1.62 $\pm$ 1.93
6,STNB,MAG,0.10 $\pm$ 0.18,4.17 $\pm$ 6.80,0.10 $\pm$ 0.16,3.76 $\pm$ 5.36,0.08 $\pm$ 0.14,1.71 $\pm$ 3.17
8,MCH,MAG,0.20 $\pm$ 0.28,9.01 $\pm$ 13.17,0.22 $\pm$ 0.40,8.56 $\pm$ 11.27,0.19 $\pm$ 0.35,4.02 $\pm$ 7.21
10,SQT,MAG,0.18 $\pm$ 0.23,8.50 $\pm$ 8.61,0.15 $\pm$ 0.21,7.55 $\pm$ 8.46,0.17 $\pm$ 0.25,4.37 $\pm$ 5.19
12,AAL,MAG,0.14 $\pm$ 0.16,6.95 $\pm$ 6.98,0.15 $\pm$ 0.18,7.51 $\pm$ 8.49,0.15 $\pm$ 0.18,3.92 $\pm$ 5.12
14,AAR,MAG,0.10 $\pm$ 0.14,5.33 $\pm$ 7.16,0.11 $\pm$ 0.14,5.58 $\pm$ 6.40,0.12 $\pm$ 0.20,3.07 $\pm$ 5.00
16,ALL,MAG,0.13 $\pm$ 0.15,6.85 $\pm$ 7.07,0.11 $\pm$ 0.11,6.74 $\pm$ 6.09,0.13 $\pm$ 0.15,3.58 $\pm$ 3.65
18,ALR,MAG,0.17 $\pm$ 0.20,9.64 $\pm$ 10.73,0.15 $\pm$ 0.22,8.20 $\pm$ 9.41,0.16 $\pm$ 0.16,4.66 $\pm$ 4.68


In [9]:
from scikit_posthocs import posthoc_dunn

for metric in ["tI", "tE", "tB"]:
    for device in ["MAG", "PZT"]:
        relative_errors_by_activity = {}
        for activity in overview_all_participants.keys():
            relative_errors_by_activity[activity] = get_relative_errors(overview_all_participants[activity][device][f"{metric} airflow (s)"], overview_all_participants[activity][device][f"{metric} (s)"])
            # normality_test(relative_errors, sensor=device, type=f"relative error {metric}")
        
        h_statistic, p_value = scipy.stats.kruskal(
            relative_errors_by_activity["SNB"],
            relative_errors_by_activity["SGB"],
            relative_errors_by_activity["MIXB"],
            relative_errors_by_activity["STNB"],
            relative_errors_by_activity["MCH"],
            relative_errors_by_activity["SQT"],
            relative_errors_by_activity["AAL"],
            relative_errors_by_activity["AAR"],
            relative_errors_by_activity["ALL"],
            relative_errors_by_activity["ALR"],
            relative_errors_by_activity["UAL"],
            relative_errors_by_activity["UAR"],
            relative_errors_by_activity["SE"],
            relative_errors_by_activity["SS"],
            relative_errors_by_activity["TR"],
        )
        print("\n")
        print(f"Kruskal-Wallis H-test test for {metric} {device}: H()={h_statistic:.1f} p={p_value:.4f}")
        
        if p_value < 0.05:
            relative_errors_list = [relative_errors_by_activity[activity].tolist() for activity in relative_errors_by_activity]
            #print(np.argwhere(posthoc_dunn(relative_errors_list, p_adjust="bonferroni") < 0.05))
            dunn = posthoc_dunn(relative_errors_list, p_adjust="bonferroni")

            for i, activity in enumerate(overview_all_participants.keys()):
                aux = np.argwhere(dunn.iloc[i, :] < 0.05)
                print(f"{activity}: {[list(overview_all_participants.keys())[a] for a in aux.flatten()]}")                
    print("\n")







Kruskal-Wallis H-test test for tI MAG: H()=47.0 p=0.0000
SNB: []
SGB: []
MIXB: []
STNB: []
MCH: []
SQT: []
AAL: []
AAR: ['SS']
ALL: []
ALR: []
UAL: []
UAR: ['SS']
SE: []
SS: ['AAR', 'UAR', 'TR']
TR: ['SS']


Kruskal-Wallis H-test test for tI PZT: H()=48.0 p=0.0000
SNB: ['SQT', 'AAL', 'SS']
SGB: []
MIXB: []
STNB: ['SQT', 'SS']
MCH: []
SQT: ['SNB', 'STNB', 'AAR']
AAL: ['SNB']
AAR: ['SQT', 'SS']
ALL: []
ALR: []
UAL: []
UAR: []
SE: []
SS: ['SNB', 'STNB', 'AAR']
TR: []




Kruskal-Wallis H-test test for tE MAG: H()=53.8 p=0.0000
SNB: []
SGB: []
MIXB: []
STNB: []
MCH: []
SQT: []
AAL: ['AAR', 'UAL', 'UAR', 'TR']
AAR: ['AAL']
ALL: []
ALR: []
UAL: ['AAL', 'SS']
UAR: ['AAL']
SE: []
SS: ['UAL']
TR: ['AAL']


Kruskal-Wallis H-test test for tE PZT: H()=26.4 p=0.0231
SNB: []
SGB: []
MIXB: []
STNB: ['AAL']
MCH: []
SQT: []
AAL: ['STNB']
AAR: []
ALL: []
ALR: []
UAL: []
UAR: []
SE: []
SS: []
TR: []




Kruskal-Wallis H-test test for tB MAG: H()=2.4 p=0.9997


Kruskal-Wallis H-test test for tB PZT: H()

### Overall RP analysis

In [10]:
# Transform overview from participant specific into all participants
overview_all = transform_overview_on_overall(overview)
overview_all.keys()

dict_keys(['MAG', 'Airflow', 'PZT'])

In [11]:
get_breath_parameters_performance(overview_all, target=None)

N=3069 tI for MAG
N=2706 tI for PZT
N=3098 tE for MAG
N=2704 tE for PZT
N=2953 tB for MAG
N=2519 tB for PZT


,Parameter,Sensor,Abs. error (s),Rel. error (%),Slope,Intercept,R^2
0,tI,MAG,0.19 $\pm$ 0.31,8.37 $\pm$ 12.04,1.01,-0.01,0.83
1,tI,PZT,0.41 $\pm$ 0.58,17.48 $\pm$ 20.33,0.92,0.19,0.52
2,tE,MAG,0.19 $\pm$ 0.30,8.11 $\pm$ 11.07,0.95,0.10,0.87
3,tE,PZT,0.41 $\pm$ 0.61,16.72 $\pm$ 19.87,0.92,0.18,0.60
4,tB,MAG,0.18 $\pm$ 0.29,4.08 $\pm$ 5.76,0.99,0.03,0.96
5,tB,PZT,0.39 $\pm$ 0.58,8.66 $\pm$ 10.74,0.97,0.15,0.84


In [12]:
import plotly.graph_objects as go

for metric in ["tI", "tE", "tB"]:
    rel_error_mag = get_relative_errors(overview_all["MAG"][f"{metric} airflow (s)"], overview_all["MAG"][f"{metric} (s)"])
    rel_error_pzt = get_relative_errors(overview_all["PZT"][f"{metric} airflow (s)"], overview_all["PZT"][f"{metric} (s)"])

    print(f"{metric}: U({len(rel_error_mag) + len(rel_error_pzt)}) {scipy.stats.mannwhitneyu(rel_error_mag, rel_error_pzt)}")

    fig = go.Figure()
    fig.add_trace(go.Violin(y=rel_error_mag, name="MAG"))
    fig.add_trace(go.Violin(y=rel_error_pzt, name="PZT"))

    # fig.show()

    #normality_test(rel_error_mag, sensor="MAG", type=f"relative error {metric}")
    #normality_test(rel_error_pzt, sensor="PZT", type=f"relative error {metric}")


tI: U(5775) MannwhitneyuResult(statistic=4169159.0, pvalue=0.7904353096202394)
tE: U(5802) MannwhitneyuResult(statistic=4074548.0, pvalue=0.07340347460361304)
tB: U(5472) MannwhitneyuResult(statistic=3686888.5, pvalue=0.5778584021630049)


### Bias and variability analysis

In [13]:
for id in overview.keys():
    for activity in overview[id].keys():
        for sensor in ["MAG", "PZT"]:
            if any(overview[id][activity][sensor]["tE (s)"] > 10):
                print(f'{overview[id][activity][sensor]["tE (s)"]} ({id} - {activity} - {sensor})')

[ 3.96 12.88  4.49  4.58] (Y6O3 - SGB - PZT)


In [14]:
from performance import get_bias_variability

get_bias_variability(overview_all_participants, target="Activity", metric="tI", relative_error=False)
get_bias_variability(overview_all, target="overview", metric="tI", relative_error=False)

,Sensor,Bias (s),Variability (s)
0,MAG,0.020,0.363
1,PZT,0.021,0.708


In [15]:
overview_all_except =  transform_overview_on_overall(overview, ignore_key="SGB")

In [16]:
activity = "SS"
for metric in ["tI", "tE", "tB"]:
        bland_altman_plot(overview_all_except["MAG"][f"{metric} (s)"], overview_all_except["MAG"][f"{metric} airflow (s)"], overview_all_except["PZT"][f"{metric} (s)"], overview_all_except["PZT"][f"{metric} airflow (s)"], metric)
        #bland_altman_plot(overview_all_participants[activity]["MAG"][f"{metric} (s)"], overview_all_participants[activity]["MAG"][f"{metric} airflow (s)"], overview_all_participants[activity]["PZT"][f"{metric} (s)"], overview_all_participants[activity]["PZT"][f"{metric} airflow (s)"], metric, activity)


MAG | mean: 0.016, lloa: -0.66, uloa: 0.69 
PZT | mean: 0.031, lloa: -1.23, uloa: 1.29 


MAG | mean: -0.003, lloa: -0.66, uloa: 0.66 
PZT | mean: 0.001, lloa: -1.28, uloa: 1.28 


MAG | mean: 0.003, lloa: -0.63, uloa: 0.64 
PZT | mean: 0.005, lloa: -1.28, uloa: 1.29 
